In [ ]:
#Heitor Lima e Silva - 202404940013

#  Árvore Rubro-Negra (Red-Black Tree)

VERMELHO = True
PRETO = False


class Node:
    def __init__(self, key):
        self.key = key
        self.color = VERMELHO
        self.left = None
        self.right = None
        self.parent = None


class RedBlackTree:

    def __init__(self):
        self.NULL = Node(None)
        self.NULL.color = PRETO
        self.root = self.NULL

    #Rotações

    def left_rotate(self, x):
        y = x.right
        x.right = y.left

        if y.left != self.NULL:
            y.left.parent = x

        y.parent = x.parent

        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y

        y.left = x
        x.parent = y

    def right_rotate(self, x):
        y = x.left
        x.left = y.right

        if y.right != self.NULL:
            y.right.parent = x

        y.parent = x.parent

        if x.parent is None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y

        y.right = x
        x.parent = y

    #Inserção

    def insert(self, key):
        new_node = Node(key)
        new_node.left = self.NULL
        new_node.right = self.NULL

        parent = None
        current = self.root

        while current != self.NULL:
            parent = current
            if new_node.key < current.key:
                current = current.left
            elif new_node.key > current.key:
                current = current.right
            else:
                return  # duplicado não inserido

        new_node.parent = parent

        if parent is None:
            self.root = new_node
        elif new_node.key < parent.key:
            parent.left = new_node
        else:
            parent.right = new_node

        self.fix_insert(new_node)

    def fix_insert(self, k):
        while k.parent and k.parent.color == VERMELHO:

            if k.parent == k.parent.parent.left:
                uncle = k.parent.parent.right

                if uncle.color == VERMELHO:
                    k.parent.color = PRETO
                    uncle.color = PRETO
                    k.parent.parent.color = VERMELHO
                    k = k.parent.parent

                else:
                    if k == k.parent.right:
                        k = k.parent
                        self.left_rotate(k)
                    k.parent.color = PRETO
                    k.parent.parent.color = VERMELHO
                    self.right_rotate(k.parent.parent)

            else:
                uncle = k.parent.parent.left

                if uncle.color == VERMELHO:
                    k.parent.color = PRETO
                    uncle.color = PRETO
                    k.parent.parent.color = VERMELHO
                    k = k.parent.parent

                else:
                    if k == k.parent.left:
                        k = k.parent
                        self.right_rotate(k)
                    k.parent.color = PRETO
                    k.parent.parent.color = VERMELHO
                    self.left_rotate(k.parent.parent)

        self.root.color = PRETO

    #Remoção

    def transplant(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v

        v.parent = u.parent

    def delete(self, key):
        z = self.search_node(self.root, key)
        if z == self.NULL:
            return False

        y = z
        y_original_color = y.color

        if z.left == self.NULL:
            x = z.right
            self.transplant(z, z.right)

        elif z.right == self.NULL:
            x = z.left
            self.transplant(z, z.left)

        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right

            if y.parent != z:
                self.transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color

        if y_original_color == PRETO:
            self.fix_delete(x)

        return True

    def fix_delete(self, x):
        while x != self.root and x.color == PRETO:

            if x == x.parent.left:
                sibling = x.parent.right

                if sibling.color == VERMELHO:
                    sibling.color = PRETO
                    x.parent.color = VERMELHO
                    self.left_rotate(x.parent)
                    sibling = x.parent.right

                if sibling.left.color == PRETO and sibling.right.color == PRETO:
                    sibling.color = VERMELHO
                    x = x.parent

                else:
                    if sibling.right.color == PRETO:
                        sibling.left.color = PRETO
                        sibling.color = VERMELHO
                        self.right_rotate(sibling)
                        sibling = x.parent.right

                    sibling.color = x.parent.color
                    x.parent.color = PRETO
                    sibling.right.color = PRETO
                    self.left_rotate(x.parent)
                    x = self.root

            else:
                sibling = x.parent.left

                if sibling.color == VERMELHO:
                    sibling.color = PRETO
                    x.parent.color = VERMELHO
                    self.right_rotate(x.parent)
                    sibling = x.parent.left

                if sibling.left.color == PRETO and sibling.right.color == PRETO:
                    sibling.color = VERMELHO
                    x = x.parent

                else:
                    if sibling.left.color == PRETO:
                        sibling.right.color = PRETO
                        sibling.color = VERMELHO
                        self.left_rotate(sibling)
                        sibling = x.parent.left

                    sibling.color = x.parent.color
                    x.parent.color = PRETO
                    sibling.left.color = PRETO
                    self.right_rotate(x.parent)
                    x = self.root

        x.color = PRETO

    #Pesquisa

    def search(self, key):
        return self.search_node(self.root, key) != self.NULL

    def search_node(self, node, key):
        while node != self.NULL:
            if key == node.key:
                return node
            if key < node.key:
                node = node.left
            else:
                node = node.right
        return self.NULL

    #Auxiliares

    def minimum(self, node):
        while node.left != self.NULL:
            node = node.left
        return node

    #Impressão

    def print_tree(self, node=None, indent="", last=True):
        if node is None:
            node = self.root

        if node != self.NULL:
            print(indent, "└── " if last else "├── ",
                  f"{node.key} ({'V' if node.color == VERMELHO else 'P'})", sep="")
            indent += "     " if last else "│    "
            self.print_tree(node.left, indent, False)
            self.print_tree(node.right, indent, True)


#MODO INTERATIVO DO TERMINAL

def menu():
    print(" ÁRVORE RUBRO-NEGRA (INTERATIVA) ")
    print("1- Inserir")
    print("2- Remover")
    print("3- Pesquisar")
    print("4- Mostrar árvore")
    print("0- Sair")
    return input("Escolha: ")


if __name__ == "__main__":
    rbt = RedBlackTree()

    while True:
        opc = menu()

        if opc == "1":
            valor = input("Digite o valor a inserir: ")
            try:
                valor = int(valor)
            except:
                pass
            rbt.insert(valor)
            print("Inserido.")

        elif opc == "2":
            valor = input("Digite o valor a remover: ")
            try:
                valor = int(valor)
            except:
                pass
            if rbt.delete(valor):
                print("Removido.")
            else:
                print("Valor não encontrado.")

        elif opc == "3":
            valor = input("Valor da pesquisar: ")
            try:
                valor = int(valor)
            except:
                pass
            print("Encontrado." if rbt.search(valor) else "Não encontrado.")

        elif opc == "4":
            print("\nÁrvore atual:")
            rbt.print_tree()

        elif opc == "0":
            print("Encerrado.")
            break

        else:
            print("Opção inválida")


 ÁRVORE RUBRO-NEGRA (INTERATIVA) 
1- Inserir
2- Remover
3- Pesquisar
4- Mostrar árvore
0- Sair
Escolha: 0
Encerrando...
